# Process Wikipidea Pageviews

In [1]:
import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import warnings
warnings.filterwarnings('ignore')
import pyspark.pandas as ps
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession
         .builder
         .master('local[*]')
         .config('spark.sql.execution.arrow.pyspark.enabled', 'true')
         .getOrCreate())
spark.sparkContext.setLogLevel('OFF')

In [ ]:
from glob import glob

## First round of parquet

In [ ]:
files = glob('/mnt/data/public/wikipedia/pageviews/pageview_complete'
             '/202[23]/*/*-user.bz2')

In [ ]:
def pageviews_parquet(file):
    from pyspark.sql import functions as F
    return (
        spark.read.csv(file, sep=' ')
        .withColumn('filename', F.input_file_name())
    )

In [ ]:
import py4j.protocol

finished = []
errors = []
for file in files:
    print(file)
    try:
        pageviews_parquet(file).write.parquet(
            'pageviews_parquet',
            mode='append'
        )
    except py4j.protocol.Py4JJavaError as e:
        print("Py4JJavaError encountered:")
        print("File skipped")
        errors.append(file)
        continue
    finished.append(file)

## Second round of parquet

In [ ]:
repeat_files = glob('/mnt/data/public/wikipedia/pageviews/pageview_complete/'
                    '2022-*/*')

In [ ]:
import py4j.protocol

finished = []
errors = []
for file in repeat_files:
    print(file)
    try:
        pageviews_parquet(file).write.parquet(
            'pageviews_parquet_repeat',
            mode='append'
        )
    except py4j.protocol.Py4JJavaError as e:
        print("Py4JJavaError encountered:")
        print("File skipped")
        errors.append(file)
        continue
    finished.append(file)